In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
from lxml import html
import pdfplumber
import csv

In [2]:
def scrapper_w1():
    url = "https://unacademy.com/content/railway-exam/study-material/general-awareness/inventions-discoveries/"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        table = soup.find("table")
        
        if table:
            header_tags = table.find_all("th")
            if header_tags:
                headers = [th.text.strip() for th in header_tags]
            else:
                headers = ["Invention/Discovery", "Name of the Inventor", "Year of Invention"]
            
            rows = []
            for tr in table.find_all("tr")[1:]: 
                cells = [td.text.strip() for td in tr.find_all("td")]
                if cells: 
                    rows.append(cells)
            
            df = pd.DataFrame(rows, columns=headers)
            return df
        else:
            print("Chart not found")
    else:
        print("Error accesing to the website", response.status_code)

In [3]:
def scrapper_w2():
    url = "https://www.ipoi.gov.ie/en/student-teacher-zone/inventions-a-z/"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        content_wrapper = soup.find("div", id="ContentWrapper")
        
        if content_wrapper:
            tables = content_wrapper.find_all("table")  

            all_data = []

            for table in tables:
                header_tags = table.find_all("th")
                if header_tags:
                    headers = [th.text.strip() for th in header_tags]
                else:
                    headers = ["Invention/Discovery", "Name of the Inventor", "Year of Invention"]
                
                rows = []
                for tr in table.find_all("tr")[1:]: 
                    cells = [td.text.strip() for td in tr.find_all("td")]
                    if cells: 
                        rows.append(cells)

                df1 = pd.DataFrame(rows, columns=headers)
                all_data.append(df1)

            if all_data:
                df1 = pd.concat(all_data, ignore_index=True)
                return df1
            else:
                print("No charts found")
        else:
            print("Container not found")
    else:
        print("Error accesing to the website", response.status_code)

In [4]:
def scrapper_w3():
    url = "https://www.adda247.com/defence-jobs/important-inventions-and-their-inventors/"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        tables = soup.find_all("table")

        all_data = []

        selected_tables = tables[1:3] if len(tables) > 2 else tables

        for table in selected_tables:
            rows = table.find_all("tr")  
            table_data = []

            for row in rows:
                cols = row.find_all(["td", "th"]) 
                cols = [col.text.strip() for col in cols]  
                if cols:  
                    table_data.append(cols)

            if table_data:
                all_data.extend(table_data)

        df2 = pd.DataFrame(all_data)
        df2.columns = ["Invention/Discovery", "Name of the Inventor", "Year of Invention"]
        return df2

    else:
        print("Error acccesing to the website", response.status_code)


In [5]:
def scrapper_w4():
    pdf_url = "https://cdn1.byjus.com/wp-content/uploads/2020/06/List-of-Important-Inventions-Discoveries.pdf"

    pdf_response = requests.get(pdf_url)
    pdf_path = "../inventions_discoveries.pdf"

    with open(pdf_path, "wb") as file:
        file.write(pdf_response.content)
    table_data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if row and len(row) == 3 and "Invention/Discovery" not in row[0]:
                        table_data.append(row)

    all_data = []
    if table_data:
        all_data.extend(table_data)

        df3 = pd.DataFrame(all_data)

    df3.columns = ["Invention/Discovery", "Name of the Inventor", "Year of Invention"]
    return df3

In [6]:
def scrapper_w5():
    url = "https://www.studyiq.com/articles/inventions-and-discoveries/"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")

        entry_content = soup.find("div", class_="entry-content")

        if not entry_content:
            print("Entry-content not found")
        else:
            tables = entry_content.find_all("table")

            if not tables or len(tables) < 3:
                print("Error with one or more charts")
            else:
                second_table = tables[1]  
                third_table = tables[2]  

                combined_data = []

                for row in second_table.find_all("tr"):
                    cols = row.find_all(["th", "td"])
                    cols = [col.text.strip() for col in cols]
                    if cols and "Invention" not in cols:  
                        combined_data.append(cols)

                first_row = True  
                for row in third_table.find_all("tr"):
                    cols = row.find_all(["th", "td"])
                    cols = [col.text.strip() for col in cols]
                    if cols:
                        if first_row:
                            first_row = False  
                            continue
                        combined_data.append(cols)

                df4 = pd.DataFrame(combined_data, columns=["","Invention/Discovery", "Name of the Inventor", "Year of Invention"])
                return df4



In [7]:
def general_file_creator():
    dataframes = {
        "df": scrapper_w1(),
        "df1": scrapper_w2(),
        "df2": scrapper_w3(),
        "df3": scrapper_w4(),
        "df4": scrapper_w5()
    }

    selected_columns = {
        "df": ["Invention/Discovery", "Name of the Inventor", "Year of Invention"],  
        "df1": ["Invention/Discovery", "Name of the Inventor", "Year of Invention"],
        "df2": ["Invention/Discovery", "Name of the Inventor", "Year of Invention"],
        "df3": ["Invention/Discovery", "Name of the Inventor", "Year of Invention"],
        "df4": ["Invention/Discovery", "Name of the Inventor", "Year of Invention"]
    }

    count = 1
    for key, df in dataframes.items():
        if key in selected_columns:
            selected_cols = selected_columns[key]
            
            filtered_df = df[selected_cols] if all(col in df.columns for col in selected_cols) else df
            
            csv_filename = f"../../raw_data/scrape_data/raw_data_balam_{count}.csv"
            count +=1
            filtered_df.to_csv(csv_filename, index=False)
            
            print(f"CSV file'{csv_filename}' saved")

general_file_creator()

CSV file'../../raw_data/scrape_data/raw_data_balam_1.csv' saved
CSV file'../../raw_data/scrape_data/raw_data_balam_2.csv' saved
CSV file'../../raw_data/scrape_data/raw_data_balam_3.csv' saved
CSV file'../../raw_data/scrape_data/raw_data_balam_4.csv' saved
CSV file'../../raw_data/scrape_data/raw_data_balam_5.csv' saved


In [9]:
df = scrapper_w1()[["Invention/Discovery", "Name of the Inventor", "Year of Invention"]]
df1 = scrapper_w2()[["Invention/Discovery", "Name of the Inventor", "Year of Invention"]]
df2 = scrapper_w3()[["Invention/Discovery", "Name of the Inventor", "Year of Invention"]]
df3 = scrapper_w4()[["Invention/Discovery", "Name of the Inventor", "Year of Invention"]]
df4 = scrapper_w5()[["Invention/Discovery", "Name of the Inventor", "Year of Invention"]]

combined_df = pd.concat([df, df1, df2, df3, df4], ignore_index=True)

combined_df = combined_df[combined_df.iloc[:, 0] != combined_df.columns[0]]

combined_df.to_csv("../../raw_data/scrape_data/raw_data_balam.csv", index=False)
print("Csv created'")

Csv created'
